<a href="https://colab.research.google.com/github/mahima1633/Medical_NER/blob/main/Clinical_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Building Custom Named Entity Recognition Model Using Spacy**

In [ ]:
import spacy

In [ ]:
# !python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

2023-12-21 06:02:32.094866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 06:02:32.094950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 06:02:32.096739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 06:02:33.569279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp

In [ ]:
doc = nlp("Narendra Modi is PM of India")

In [ ]:
doc

Narendra Modi is PM of India

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc.ents

(Narendra Modi, India)

In [ ]:
doc.ents[0], type(doc.ents[0])

(Narendra Modi, spacy.tokens.span.Span)

In [ ]:
doc.ents[1], type(doc.ents[1])

(influenza, spacy.tokens.span.Span)

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
import json
# https://www.kaggle.com/datasets/finalepoch/medical-ner
with open('/content/Corona2.json', 'r') as f:
    data = json.load(f)

In [ ]:
data['examples'][0]

In [ ]:
data['examples'][0].keys()

In [ ]:

data['examples'][0]['content']

In [ ]:

data['examples'][0]['annotations'][0]

In [ ]:
training_data = []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

In [ ]:
training_data[0]['text']

In [ ]:
training_data[0]['entities']

In [ ]:
training_data[0]['text'][360:371]

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

In [ ]:
# https://spacy.io/usage/training#quickstart

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

2023-12-21 05:16:47.100948: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 05:16:47.101023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 05:16:47.102547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 05:16:48.527527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['to

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nlp_ner = spacy.load("model-best")

In [ ]:
doc = nlp_ner("  Medicine, diseases, and pathogens play integral roles in the field and psoriasis. These medications, typically administered by injection, target specific molecules involved in the inflammatory process, providing more targeted and effective therapy.In the realm of mental health, selective serotonin reuptake inhibitors (SSRIs), such as fluoxetine and sertraline, are commonly prescribed for conditions like depression and anxiety. These medications work by increasing the levels of serotonin, a neurotransmitter associated with mood regulation, in the brain.Corticosteroids, such as hydrocortisone, find use in various dermatological conditions, including eczema and psoriasis. Topical corticosteroids help reduce inflammation and alleviate symptoms like itching and redness when applied to the skin. of healthcare. Medicines, also known as pharmaceuticals, are substances designed to prevent, diagnose, treat, or cure diseases. They can range from over-the-counter drugs to prescription medications and are often formulated to target specific pathways or mechanisms within the body. Diseases, on the other hand, encompass a broad spectrum of abnormal conditions affecting the body's structure or function, leading to symptoms or signs of illness. They can be caused by various factors, including pathogens.Pathogens are microorganisms such as bacteria, viruses, fungi, and parasites that cause infectious diseases. Bacteria are single-celled organisms, viruses are genetic material surrounded by a protein coat, fungi are multicellular organisms, and parasites are organisms that live on or in a host organism. Understanding the interactions between medicines, diseases, and pathogens is crucial for effective healthcare. Antibiotics, for instance, are commonly prescribed to combat bacterial infections, while antiviral drugs target viral infections.In some cases, diseases are caused by an overactive immune system, leading to conditions like autoimmune diseases. Medications such as immunosuppressants are prescribed to manage these conditions by suppressing the immune response. Vaccines, another category of medicines, stimulate the immune system to provide protection against specific pathogens, preventing diseases like influenza or measles.In conclusion, the interconnected web of medicines, diseases, and pathogens underscores the complexity of healthcare. Ongoing research and advancements in medical science continue to expand our understanding In the field of infectious diseases, antiviral medications like oseltamivir (Tamiflu) play a crucial role in treating influenza. These drugs inhibit the replication of the influenza virus, helping to alleviate symptoms and shorten the duration of the illness when taken early in the course of the infection.Opioids, like oxycodone and morphine, are potent pain relievers often prescribed for severe pain, such as that resulting from surgery or cancer. However, the opioid epidemic has highlighted the risks of dependency and overdose associated with these medications, prompting increased scrutiny and efforts to find alternative pain management strategies.Autoimmune diseases, such as lupus and multiple sclerosis, often require immunosuppressive medications like methotrexate or azathioprine.hese drugs help modulate the overactive immune response responsible for attacking the body's own tissues.Biologic drugs, like adalimumab and infliximab, have revolutionized the treatment of conditions such as rheumatoid arthritis  and ability to combat various health challenges, promoting the well-being of individuals and communities worldwide.")
colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#a6e22d"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)